# COBS File Analyzer for BBFRAMEs

In [91]:
from cobs import cobs   # https://github.com/cmcqueen/cobs-python

bbf_fname = 'bbc/test.bbc'
a_fname = 'analyze/test.txt'

In [92]:
def add_to_histogram(hist, key):
    if key in hist:
        hist[key] += 1
    else:
        hist[key] = 1

def report_histogram(hist, file, title):
    print(title, file=file)
    print('='*len(title), file=file)
    for (k,v) in hist.items():
        print(f'{k}   {v}', file=file)
    print('', file=file)

In [93]:
modewords = dict()      # key = modeword as bytes, value = count
framelengths = dict()   # key = frame length in bytes, value = count

def analyze_modeword_bbframe(buf, offset):
    # print(f"BBFRAME at offset {offset} length {len(buf)}")
    add_to_histogram(modewords, buf[0:4])
    add_to_histogram(framelengths, len(buf) - 4)

In [94]:
frame_count = 0
with open(bbf_fname, 'rb') as f:
    buf = f.read()
    start = 1   # skip over the extra framing byte at the beginning for cleaner histograms
    for end in range(1, len(buf)):
        if buf[end] == 0:
            analyze_modeword_bbframe(cobs.decode(buf[start:end])[:-1], offset=start)
            start = end + 1
            frame_count += 1

with open(a_fname, 'w') as of:
    print(f'Found {frame_count} frames in {len(buf)} bytes.\n', file=of)
    report_histogram(modewords, of, 'Modeword Histogram')
    report_histogram(framelengths, of, 'Frame Length Histogram')